Will basically import pretrained model till the flatten layer and freeze its weight. The output after flatten layer are embeddings.

In [1]:
!pip install tensorflow
!pip install numpy

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.

In [2]:
import tensorflow as tf
import numpy as np

2024-07-18 13:14:36.357764: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 13:14:36.358713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 13:14:36.360808: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 13:14:36.370130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 13:14:36.386133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# Create model (img -> embeds)

In [3]:
# now import model

conv_base = tf.keras.applications.MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)
)

conv_base.trainable = False

#since this doesnt include Flattening layer, add

model = tf.keras.Sequential([
    conv_base, 
    tf.keras.layers.GlobalAveragePooling2D()
])

# Take images, create embeddings

Since cromadb takes in an array of embeddings, and array of chunks, we'll create an array of both

In [4]:
#now create embeddings for an image
!pip install opencv-python

import cv2

def create_embeds(img_path):
    #will take an image path and return and array of embeddings
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255.0
    img = np.expand_dims(img, axis = 0)

    embed = model.predict(img)
    return embed[0]

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
img_path_array = []
img_embeds_array = []

In [6]:
#take images from 'flowers' folder
import glob


for img in glob.glob("flowers/*/*.jpg")[:20]:                   #selected only 20 images for now, for quicker training
    img_path_array.append(img)
    img_embeds_array.append(list(create_embeds(img)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


# Store embeddings

In [7]:
img_embeds_array = np.array(img_embeds_array).astype(float)    #since the embeddings are expected to be of type "float"

In [9]:
!pip install chromadb
import chromadb
import os

client = chromadb.PersistentClient(path = os.getcwd())
collection = client.get_or_create_collection("image_embeddings")

collection.add(
    documents = img_path_array,
    embeddings = img_embeds_array,
    ids = ["id" + str(i) for i in range(len(img_embeds_array))]
)

  Using cached chromadb-0.5.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached chroma_hnswlib-0.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.111.1-py3-none-any.whl.metadata (26 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.18.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.3 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.46b0-py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached tokenizers-0.19.1-cp310-cp

# Create embeds for input img and do simsearch

In [15]:
ip_img = create_embeds("images.jpeg")
ip_img = [list(ip_img)]
ip_img = np.array(ip_img).astype(float)

sim_imgs = collection.query(
    query_embeddings = ip_img,
    n_results = 5
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [16]:
sim_imgs

{'ids': [['id3', 'id0', 'id5', 'id1', 'id4']],
 'distances': [[758.3054944500068,
   784.9338626936394,
   801.7721477713077,
   814.7666359934615,
   816.1960615797799]],
 'metadatas': [[None, None, None, None, None]],
 'embeddings': None,
 'documents': [['flowers/dandelion/18183515403_13a9ca6d86_n.jpg',
   'flowers/dandelion/7469617666_0e1a014917.jpg',
   'flowers/dandelion/17688233756_21879104c1_n.jpg',
   'flowers/dandelion/14576445793_582aa6446b_m.jpg',
   'flowers/dandelion/2490828907_5094017933_m.jpg']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [18]:
sim_imgs["documents"]                   #this contains the list of paths of the related images

[['flowers/dandelion/18183515403_13a9ca6d86_n.jpg',
  'flowers/dandelion/7469617666_0e1a014917.jpg',
  'flowers/dandelion/17688233756_21879104c1_n.jpg',
  'flowers/dandelion/14576445793_582aa6446b_m.jpg',
  'flowers/dandelion/2490828907_5094017933_m.jpg']]